In [7]:
%pip install duckdb

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.8/12.3 MB 11.2 MB/s eta 0:00:01
   ----------------- ---------------------- 5.5/12.3 MB 13.4 MB/s eta 0:00:01
   --------------------------- ------------ 8.4/12.3 MB 13.7 MB/s eta 0:00:01
   -------------------------------------- - 11.8/12.3 MB 14.2 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 13.6 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Célula 1
import duckdb
import pandas as pd
from pathlib import Path

PARQUET_PATH = Path(r"C:\Users\Vitoria Oliveira\Desktop\bigdata_dp\aps\emissoes_gee\emissoes_por_decada_streaming.parquet")
df = pd.read_parquet(PARQUET_PATH)
conn = duckdb.connect(database=':memory:')
conn.register('emissoes', df)

In [16]:
# ve quais colunas eu tenho
conn.execute("PRAGMA table_info('emissoes');").df()

,cid,name,type,notnull,dflt_value,pk
0,0,decada,VARCHAR,False,None,False
1,1,Setor_Agrupado,VARCHAR,False,None,False
2,2,Cidade,VARCHAR,False,None,False
3,3,Gás,VARCHAR,False,None,False
4,4,Bioma,VARCHAR,False,None,False
5,5,emissao_t,DOUBLE,False,None,False


In [ ]:
# ve quanto de emissão total por década
conn.execute("""
SELECT 
    decada,
    SUM(emissao_t) as emissao_total
FROM emissoes
GROUP BY decada
ORDER BY decada
""").df()

,decada,emissao_total
0,1970s,1.017847e+10
1,1980s,1.151769e+10
2,1990s,7.702860e+10
3,2000s,6.755515e+10
4,2010s,3.115889e+10
5,2020s,1.738720e+10


In [21]:
# ve quanto cada cidade emitiu
conn.execute("""
SELECT 
    Cidade,
    SUM(emissao_t) as emissao_total,
FROM emissoes
GROUP BY Cidade
ORDER BY Cidade ASC
""").df()

,Cidade,emissao_total
0,Abadia de Goiás (GO),5.570210e+06
1,Abadiânia (GO),3.980967e+07
2,Abaetetuba (PA),5.852624e+07
3,Abaiara (CE),1.890590e+06
4,Abatiá (PR),4.100681e+06
...,...,...
3067,Álvaro de Carvalho (SP),3.213466e+06
3068,Áurea (RS),4.296056e+06
3069,Ângulo (PR),1.609915e+06
3070,Óbidos (PA),5.864008e+07


In [ ]:
# quanto cada setor emitiu
conn.execute("""
SELECT 
    Setor_Agrupado,
    SUM(emissao_t) as emissao_total,
    ROUND(SUM(emissao_t) * 100.0 / (SELECT SUM(emissao_t) FROM emissoes), 2) as percentual
FROM emissoes
GROUP BY Setor_Agrupado
ORDER BY emissao_total DESC
LIMIT 10
""").df()

,Setor_Agrupado,emissao_total,percentual
0,Mudança de Uso do Solo,1.318094e+11,61.36
1,Energia e Combustíveis,4.041248e+10,18.81
2,Agropecuária,3.339637e+10,15.55
3,Processos Industriais,6.670353e+09,3.11
4,Resíduos,2.537423e+09,1.18


In [24]:
# ver o quanto cada setor emitiu em cada decada
conn.execute("""
SELECT
    decada,
    Setor_Agrupado,
    SUM(emissao_t) as emissao_total
FROM emissoes
GROUP BY decada, Setor_Agrupado
ORDER BY Setor_Agrupado, decada
""").df()

,decada,Setor_Agrupado,emissao_total
0,1970s,Agropecuária,3.726920e+09
1,1980s,Agropecuária,5.004822e+09
2,1990s,Agropecuária,7.757837e+09
3,2000s,Agropecuária,7.564222e+09
4,2010s,Agropecuária,6.592809e+09
5,2020s,Agropecuária,2.749762e+09
6,1970s,Energia e Combustíveis,5.640231e+09
7,1980s,Energia e Combustíveis,5.457033e+09
8,1990s,Energia e Combustíveis,6.431295e+09
9,2000s,Energia e Combustíveis,8.196754e+09


In [14]:
# Célula 4
conn.execute("""
SELECT 
    decada,
    Setor_Agrupado,
    SUM(emissao_t) as emissao_total
FROM emissoes
GROUP BY decada, Setor_Agrupado
ORDER BY decada, emissao_total DESC
""").df()

,decada,Setor_Agrupado,emissao_total
0,1970s,Energia e Combustíveis,5.640231e+09
1,1970s,Agropecuária,3.726920e+09
2,1970s,Processos Industriais,6.833153e+08
3,1970s,Resíduos,1.280063e+08
4,1970s,Mudança de Uso do Solo,0.000000e+00
5,1980s,Energia e Combustíveis,5.457033e+09
6,1980s,Agropecuária,5.004822e+09
7,1980s,Processos Industriais,8.347400e+08
8,1980s,Resíduos,2.210964e+08
9,1980s,Mudança de Uso do Solo,0.000000e+00


In [15]:
# Célula 5
conn.execute("""
SELECT 
    Cidade,
    SUM(emissao_t) as emissao_total,
    COUNT(DISTINCT decada) as num_decadas
FROM emissoes
WHERE Cidade IS NOT NULL
GROUP BY Cidade
ORDER BY emissao_total DESC
LIMIT 20
""").df()

,Cidade,emissao_total,num_decadas
0,NA (NA),2.030709e+10,6
1,NA (SP),5.590600e+09,6
2,São Félix do Xingu (PA),4.911190e+09,6
3,Juara (MT),2.834464e+09,6
4,Colniza (MT),2.200245e+09,6
5,Novo Repartimento (PA),2.140946e+09,6
6,Cumaru do Norte (PA),2.131269e+09,6
7,Marabá (PA),2.130581e+09,6
8,Vila Bela da Santíssima Trindade (MT),2.074550e+09,6
9,Novo Progresso (PA),1.884832e+09,6
